In [2]:
import itertools
import yaml
import sys
import os
from utils_et.utils import make_dict_from_table
trial_config=yaml.load(open('./configs/trial_config.yaml'), Loader=yaml.Loader)
sys.path.insert(0, trial_config['path_params']['cprd_package_paths'])
cprd_config=yaml.load(open(trial_config['path_params']['cprd_package_paths']+'/config/config.yaml'), Loader=yaml.Loader)
from CPRD.functions import MedicalDictionary,tables,modalities
from CPRD.config.spark import spark_init,read_txt,read_parquet
import pyspark.sql.functions as F
import pandas as pd
import logging
from itertools import chain
from datetime import date



os.environ["JAVA_HOME"] = trial_config['path_params']['java_home']

In [403]:
#!ipython nbconvert utils_et/utils.ipynb --to python

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook utils_et/utils.ipynb to python
[NbConvertApp] Writing 294 bytes to utils_et/utils.py


In [404]:
spark = spark_init(trial_config['pyspark_params'])

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=ehr, master=local[*]) created by __init__ at /home/shared/mo/HFclusters/dependencies/CPRD/CPRD/config/spark.py:34 

In [3]:

md=MedicalDictionary.MedicalDictionaryBase(cprd_config["file_path"],spark)

med2read = tables.retrieve_med2read_map(dir=cprd_config["file_path"]['med2read'], spark=spark)
read2icd = tables.retrieve_read2icd_map(dir=cprd_config["file_path"]['read2icd'], spark=spark)

#takes a few minutes
merged_codes=med2read.join(read2icd,med2read.readcode.contains(read2icd.read),how='left').filter(F.col('ICD').isNotNull()).cache()

#construct the icd2read dictionary
#this generates the higher level read codes so duplicates may exist. but the icd2medcode should be used for all intents and purposes. icd2read is just for sanity check or unless I only have the readcodes. 
icd2read= make_dict_from_table(merged_codes,key='ICD',value='read')
icd2medcode= make_dict_from_table(merged_codes,key='ICD',value='medcode')

In [4]:
#additional user defined phenotypes and automated phenotypes

# First check if the codes exist in the dictionary: [icdcode for icdcode in icd2medcode.keys() if icdcode.startswith('R58')]

icd10codes_pacemaker=['Z950']
medcodes_pacemaker=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_pacemaker]))

icd10codes_valve=['Z952','Z953','Z954']
medcodes_valve=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_valve]))

icd10codes_aorticstenosis=['I350','I352']
medcodes_aorticstenosis=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_aorticstenosis]))

icd10codes_corArtBypGraf=['Z951','Z955']
medcodes_corArtBypGraf=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_corArtBypGraf]))

icd10codes_anySevereBleeding='S064, S065, S066, I850, I983A, K226, K250, K252, K254, K256, K260, K262, K264, K266, K270, K272, K274, K276,\
 K280, K284, K290, K625, K661, K920, K921, K922, H431, N028, N020, N027, N023, N021, N029, N022, N025, N026, R58, T810, D62'.split(', ')+\
['R048', 'R042', 'R040', 'R041']
medcodes_anySevereBleeding=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_anySevereBleeding]))

icd10codes_mentalOrBehavioural=[icdcode for icdcode in icd2medcode.keys() if icdcode.startswith('F')]
medcodes_mentalOrBehavioural=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_mentalOrBehavioural]))

icd10codes_dialysis=['Z491','Z492']
medcodes_dialysis=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_dialysis]))

icd10codes_lungdisease=[icdcode for icdcode in icd2medcode.keys() if \
                        (icdcode.startswith('J4') | icdcode.startswith('J6') | icdcode.startswith('J7') \
                         | icdcode.startswith('J8') | icdcode.startswith('J9'))]
medcodes_lungdisease=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_lungdisease]))

icd10codes_alcoholism='E244, G312, G621, I426, K292, O354, P043, Q860, Z502, Z714'.split(', ')+ \
['F100','F108','F105','F107','F101','F103','F106','F104','F102','F109']+\
['K701', 'K709', 'K704', 'K703', 'K709D', 'K700', 'K702']+\
['T510', 'T512', 'T513', 'T518', 'T519', 'T511']+\
['Y903','Y902','Y904','Y905','Y909','Y900','Y908','Y901','Y907','Y906']+\
['Y919', 'Y910', 'Y912', 'Y913', 'Y911']
medcodes_alcoholism=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_alcoholism]))

icd10codes_muscConTissDisorder=[icdcode for icdcode in icd2medcode.keys() if icdcode.startswith('M')]
medcodes_muscConTissDisorder=list(itertools.chain.from_iterable([icd2medcode[icdcode] for icdcode in icd10codes_muscConTissDisorder]))

diagnoses_dict1={'pacemaker':{'source':'self defined',
                             'phenotypes':{'Medcode':medcodes_pacemaker,
                                           'ICD':icd10codes_pacemaker,
                                           'OPCS':['K600','K601','K602','K603','K606','K611','K612']}},
                'heart valve surgery':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_valve,
                                           'ICD':icd10codes_valve,
                                           'OPCS':['K180', 'K185', 'K254', 'K258', 'K259', 'K260', 'K261', 'K270', 'K280', 'K350', 'K358']}},
                'aortic stenosis':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_aorticstenosis,#not granular enough as they indicate all valve replacements
                                           'ICD':icd10codes_aorticstenosis,#same as above
                                           'OPCS':[]}},
                'percutaneous coronary intervention':{'source':'self defined',
                                           'phenotypes':{'Medcode':[],#not granular enough as they indicate all valve replacements
                                           'ICD':[],#same as above
                                           'OPCS':['K490','K491','K492']}},
                'coronary artery bypass graft':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_corArtBypGraf,#not granular enough as they indicate all valve replacements
                                           'ICD':icd10codes_corArtBypGraf,#same as above
                                           'OPCS':['K410','K418','K441','K460','K461']}},
                'any severe bleed except intracranial':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_anySevereBleeding,#not granular enough as they indicate all valve replacements
                                           'ICD':icd10codes_anySevereBleeding,#same as above
                                           'OPCS':[]}},
                'mental or behavioral disorder':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_mentalOrBehavioural,#not granular enough as they indicate all valve replacements
                                           'ICD':icd10codes_mentalOrBehavioural,#same as above
                                           'OPCS':[]}},
                'dialysis':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_dialysis,
                                           'ICD':icd10codes_dialysis,
                                           'OPCS':['C671','L742','L911']}},
                'lung disease':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_lungdisease,
                                           'ICD':icd10codes_lungdisease,
                                           'OPCS':[]}},
                'alcoholism':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_alcoholism,
                                           'ICD':icd10codes_alcoholism,
                                           'OPCS':[]}},
                'Musculoskeletal or connective tissue':{'source':'self defined',
                                           'phenotypes':{'Medcode':medcodes_muscConTissDisorder,
                                           'ICD':icd10codes_muscConTissDisorder,
                                           'OPCS':[]}}
               }


cancer_phenotypes=[ent for ent in md.diagDict.keys() if (('malignancy' in ent) | ('malignant' in ent))]

diagnoses_dict2={'hypertension':{'source':'existing phenotypes','phenotypes':['hypertension']},
               'diabetes':{'source':'existing phenotypes','phenotypes':['diabetes']},
                'IBD':{'source':'existing phenotypes','phenotypes':["ulcerative colitis","crohn's disease"]},
                'myocardial infarction':{'source':'existing phenotypes','phenotypes':['myocardial infarction']},
                'peripheral arterial disease':{'source':'existing phenotypes','phenotypes':['peripheral arterial disease']},
                'atrial fibrillation':{'source':'existing phenotypes','phenotypes':['atrial fibrillation']},
                'ischaemic stroke or transient attack':{'source':'existing phenotypes','phenotypes':['ischaemic stroke','transient ischaemic attack']},
                'malignant cancer':{'source':'existing phenotypes','phenotypes':cancer_phenotypes},
                'gout':{'source':'existing phenotypes','phenotypes':['gout']}}
diagnoses_dict={**diagnoses_dict1,**diagnoses_dict2}

In [16]:
with open('./configs/additional_phenotypes.yaml', 'w') as file:
    yaml.dump(diagnoses_dict, file)

In [11]:
#diagnoses_dict2=yaml.load(open('./configs/additional_phenotypes.yaml'), Loader=yaml.Loader)

# Generate Medication Phenotypes

In [25]:
prod =pd.read_table(cprd_config['file_path']['prod2bnf'], encoding='latin-1')[['prodcode', 'productname','drugsubstance','strength', 'bnfcode', 'bnfchapter']]
prod['bnf'] = prod.bnfcode.apply(lambda x: x[0:7])

In [26]:
med_phenotypes=dict()

In [136]:
all_previous_codes=[]
for med in ["Aldosterone Antagonist","Aldosterone Antagonist","ARB","ACE","BB","Digoxin","Other Diuretics","Antiplatelet","Anticoagulants","Statin"]:#"Aldosterone Antagonist","ARB","ACE","BB","Digoxin"     
    if med=="Aldosterone Antagonist":
        #inspra is an epleronone, Aldactone and Diatensec are a spironalactone prod['bnf']=='0202030'
        prod_df=prod[(prod["bnfchapter"].str.contains("Aldosterone Antagonist")) & (prod["productname"].str.contains("Eplerenone")\
                                                  |  prod["productname"].str.contains("Spir")\
                                                  |prod["productname"].str.contains("Aldactone")\
                                                  | prod["productname"].str.contains("Inspra")\
                                                  | prod["productname"].str.contains("Diatensec")
                                 )]
    elif med=="ARB":
        prod_df=prod[(prod["bnfchapter"]=="Angiotensin-ii Receptor Antagonists") | \
          (prod["bnfchapter"]=="Angiotensin-ii Receptor Antagonists With Diuretic") |\
          (prod["bnfchapter"]=="Angiotensin-ii Receptor Antagonists/Calcium-channel Blockers") |\
          (prod["bnfchapter"]=="Thiazides And Related Diuretics/Angiotensin-ii Receptor Antagonists With Diuretic") |\
          (prod["bnfchapter"]=='Thiazides And Related Diuretics/Angiotensin-ii Receptor Antagonists')]
    elif med=="ACE":
        prod_df=prod[(prod["bnfchapter"]=="Angiotensin-converting Enzyme Inhibitors") |\
          (prod["bnfchapter"]=="Thiazides And Related Diuretics/Angiotensin-converting Enzyme Inhibitors") |\
          (prod["bnfchapter"]=='Thiazides And Related Diuretics/Drugs Affecting The Renin-angiotensin System/Angiotensin-converting Enzyme Inhibitors') |\
          (prod["bnfchapter"]=="Angiotensin-converting Enzyme Inhibitors/Calcium-channel Blockers")      
         ]
    elif med=="BB":
        prod_df=prod[(prod['bnfchapter'].str.contains('Beta-adrenoceptor Blocking Drugs'))]
    elif med=='Digoxin':
        prod_df=prod[(prod["productname"].notna())&(prod["productname"].str.contains("Digoxin"))]
    elif med=='Other Diuretics':
        prod_df=prod[(prod["bnfchapter"].str.contains("Diuretic"))]
    elif med=='Antiplatelet':
        prod_df=prod[(prod["bnfchapter"].str.contains("Antiplatelet"))]
    elif med=='Anticoagulants':
        prod_df=prod[(prod["bnfchapter"].str.contains("Oral Anticoagulants"))]
    elif med=="Statin":
        prod_df=prod[(prod["bnfchapter"].str.contains("Statin"))]
    
    
    prodcode_not_previously_captured=set(prod_df["prodcode"].values)-set(all_previous_codes)
    prod_df=prod_df[prod_df["prodcode"].isin(prodcode_not_previously_captured)]
    med_phenotypes[med]={'bnf_code_list':prod_df["bnfcode"].unique().tolist(),
                         'prodcode':prod_df["prodcode"].unique().tolist()}
    all_previous_codes+=prod_df["prodcode"].unique().tolist()

In [139]:
with open('./configs/drug_phenotypes.yaml', 'w') as file:
    yaml.dump(med_phenotypes, file)